In [2]:
import pandas as pd
from pathlib import Path
import re

In [ ]:
raw = Path(r"path").read_text(encoding="utf-8", errors="replace")
lines=raw.splitlines()
print("Total lines:",len(lines))

Total lines: 99715


## WhatsApp Message Boundary Detection

WhatsApp exports store chat data as plain text, where each message begins with a
system-generated timestamp (date and time), followed by the sender name and message content.

To reliably split the raw text into individual message blocks, a regular expression
is used to detect these system-generated timestamps. This ensures that only actual
WhatsApp message headers are used as boundaries, and not user-entered dates that may
appear inside message content.

### Regular Expression for Message Boundary Detection
The following regular expression is designed to match WhatsApp message timestamps:

```regex    
/^\d{1,2}[\/]\d{1,2}[\/]\d{2},\s*\d{1,2}:\d{1,2}/gm
```

### Block Formation Logic

- Each time a line matching the WhatsApp timestamp pattern is encountered,
  it indicates the start of a new message.
- Lines following this timestamp belong to the same message until the next
  timestamp is found.
- A temporary buffer (`temp`) is used to collect lines for the current message.
- When a new timestamp is detected, the collected lines are joined and stored
  as a complete message block.
- This process continues until all lines in the text have been processed,
  ensuring that each message is accurately captured.

### Why This Approach

- Ensures message boundaries are detected reliably using system-generated data
- Avoids incorrect splitting on user-entered dates or numbers
- Preserves multi-line messages as a single logical unit
- Allows downstream parsing (DLR detection, tower extraction, activity parsing)
  to operate on clean message-level input
By using this method, we can effectively parse WhatsApp chat exports into
individual messages for further analysis or processing.

In [4]:
msg_start=re.compile(r'^\d{1,2}[\/]\d{1,2}[\/]\d{2},\s*\d{1,2}:\d{1,2}') #r"^\d{1,2}[\/]\d{1,2}[\/]\d{2},\s*\d{1,2}:\d{1,2}"g

blocks=[]
temp=[]

for line in lines:
    if msg_start.match(line):
        if temp:
            blocks.append("\n".join(temp))
        temp=[line]  
    else:
        temp.append(line) 
if temp:
    blocks.append("\n".join(temp))

print("total message blocks:",len(blocks))

total message blocks: 9502


## Filtering Relevant DLR Messages

The WhatsApp group used for this project follows a consistent reporting
convention: **every Daily Labour Report message contains the keyword `DLR`**
in the message text.

This assumption allows reliable identification of DLR-related messages
without requiring complex natural language processing.


In [5]:
dlr_blocks=[i for i in blocks if "DLR" in i.upper()]
print('DLR blocks=',len(dlr_blocks))

DLR blocks= 6351


## Extracting the Report Date
Each WhatsApp message starts with a system-generated timestamp in the
dd/mm/yy format. This step extracts the report date only from this
timestamp and ignores any dates manually entered within the message body.

Using the WhatsApp-generated date ensures consistency and avoids ambiguity
caused by multiple or inconsistent date formats
The extracted date represents the date on which the Daily Labour Report
was submitted in the WhatsApp group

In [6]:
from datetime import datetime
import re

date_re = re.compile(r'^\d{1,2}/\d{1,2}/\d{2}')

def extract_date(text):
    m = date_re.search(text)
    if not m:
        return None

    date_str = m.group(0)

    # Try day-first, then month-first
    for fmt in ("%d/%m/%y", "%m/%d/%y"):
        try:
            return datetime.strptime(date_str, fmt).date()
        except ValueError:
            continue

    return None


In [7]:
tower_re = re.compile(r'(?:Tower|T)\s*[-:]?\s*(\d{1,2})', re.I)

## Activity Normalization

Labour activities in the Daily Labour Reports appear in many inconsistent
formats due to spelling variations, abbreviations, and contractor-specific
naming. To handle this, activity text is normalized using an intent-based
rule system.

Normalization Approach

Raw activity text is first cleaned by converting it to lowercase and
removing special characters.

Non-relevant or unsupported entries (e.g. unskilled labour, totals,
concrete quantities) are filtered out early.

Remaining entries are mapped to canonical activity categories using
keyword-based intent rules rather than exact string matching.

In [8]:
def clean_text(t):
    return (
        t.lower()
         .replace("&", " ")
         .replace("-", " ")
         .replace("_", " ")
         .strip()
    )

ACTIVITY_RULES = {
    "al_carp": ["carp", "carpenter", "formwork", "shuttering", "mivan", "aluminium"],
    "rebar": ["rebar", "reinforcement", "barbender", "steel"],
    "masonry": ["mason", "masonry"],
    "waterproofing": ["waterproof", "wetshield"],
    "gypsum": ["gypsum"],
    "tiling": ["tile"],
    "railing": ["railing", "hand rail"],
    "painting": ["paint"],
}
IGNORE_WORDS = [
    "unskilled", "helper", "khalasi", "welder", "surveyor",
    "total", "date", "qty", "cum", "concrete",
    "housekeeping", "safety", "coverage"
]

def normalize_activity(text):
    t = clean_text(text)

    # ignore noise
    if any(w in t for w in IGNORE_WORDS):
        return None

    for canon, keys in ACTIVITY_RULES.items():
        if any(k in t for k in keys):
            return canon

    return None


## Activity Line Parsing

Labour activity lines in the DLR messages appear in multiple formats such as
numbered lists, symbol-separated entries, or space-separated values.
This regular expression is used to consistently extract the activity name and
labour quantity from each line

In [9]:
activity_re = re.compile(r'^\s*(?:\d+\s*[.)]?\s*)?([A-Za-z &]+?)\s*(?:[-:=]|\s+)\s*(\d+)\s*[a-zA-Z]*\s*$', re.I)



Each Daily Labour Report message is handled one at a time and converted into
clean, structured rows.

First, the message is split into individual lines. The report date is extracted
once from the WhatsApp timestamp and reused for all entries in that message.

As the code reads through the lines, it keeps track of the current tower.
Whenever a tower name appears, it becomes the active context. Any labour
entries that follow are assumed to belong to that tower until a new tower is
found.

For each activity line:

The activity name and labour count are extracted.

The activity name is normalized to a standard category.

Irrelevant or unsupported activities are skipped.

Valid entries are stored as simple records containing the date, tower, activity,
and labour count. All records are finally combined into a pandas DataFrame.

In [10]:
records = []

for blk in dlr_blocks:
    lines = blk.splitlines()

    report_date = extract_date(blk)
    current_tower = None

    for line in lines:
        mt = tower_re.search(line)
        if mt:
            current_tower = f"Tower-{mt.group(1)}"
            continue

        m = activity_re.match(line)
        if not m or not current_tower:
            continue

        raw_act = m.group(1)
        qty = int(m.group(2))

        activity = normalize_activity(raw_act)
        if not activity:
            continue

        records.append({
            "date": report_date,
            "tower": current_tower,
            "activity": activity,
            "labour_count": qty
        })

df = pd.DataFrame(records)


In [11]:
df.tail()

,date,tower,activity,labour_count
14641,2025-04-12,Tower-14,railing,0
14642,2025-04-12,Tower-14,tiling,20
14643,2025-04-12,Tower-14,al_carp,0
14644,2025-04-12,Tower-14,masonry,0
14645,2025-04-12,Tower-14,masonry,0


In [12]:
df.head()

,date,tower,activity,labour_count
0,2024-01-06,Tower-9,rebar,18
1,2024-01-06,Tower-10,al_carp,10
2,2024-01-06,Tower-10,rebar,20
3,2024-01-06,Tower-11,al_carp,40
4,2024-01-06,Tower-12,rebar,8
